## Exercises


<a id='np-ex1'></a>

### Exercise 1

This exercise uses matrix operations that arise in certain problems,
including when dealing with linear stochastic difference equations

If you aren’t familiar with all the terminology don’t be concerned – you can
skim read the background discussion and focus purely on the matrix exercise

With that said, consider the stochastic difference equation


<a id='equation-ja-sde'></a>
$$
X_{t+1} = A X_t + b + \Sigma W_{t+1} \tag{1}
$$

Here

- $ X_t, b $ and $ X_{t+1} $ are $ n \times 1 $  
- $ A $ is $ n \times n $  
- $ \Sigma $ is $ n \times k $  
- $ W_t $ is $ k \times 1 $ and $ \{W_t\} $ is iid with zero mean and variance-covariance matrix equal to the identity matrix  


Let $ S_t $ denote the $ n \times n $ variance-covariance matrix of $ X_t $

Using the rules for computing variances in matrix expressions, it can be shown from [(1)](#equation-ja-sde) that $ \{S_t\} $ obeys


<a id='equation-ja-sde-v'></a>
$$
S_{t+1} = A S_t A' + \Sigma \Sigma' \tag{2}
$$

It can be shown that, provided all eigenvalues of $ A $ lie within the unit circle, the sequence $ \{S_t\} $ converges to a unique limit $ S $

This is the **unconditional variance** or **asymptotic variance** of the stochastic difference equation

As an exercise, try writing a simple function that solves for the limit $ S $ by iterating on [(2)](#equation-ja-sde-v) given $ A $ and $ \Sigma $

To test your solution, observe that the limit $ S $ is a solution to the matrix equation


<a id='equation-ja-dle'></a>
$$
S = A S A' + Q
\quad \text{where} \quad Q := \Sigma \Sigma' \tag{3}
$$

This kind of equation is known as a **discrete time Lyapunov equation**

The [QuantEcon package](http://quantecon.org/julia_index.html)
provides a function called `solve_discrete_lyapunov` that implements a fast
“doubling” algorithm to solve this equation

Test your iterative method against `solve_discrete_lyapunov` using matrices

$$
A =
\begin{bmatrix}
    0.8 & -0.2  \\
    -0.1 & 0.7
\end{bmatrix}
\qquad
\Sigma =
\begin{bmatrix}
    0.5 & 0.4 \\
    0.4 & 0.6
\end{bmatrix}
$$

### Exercise 2

Take a stochastic process for $ \{y_t\}_{t=0}^T $

$$
y_{t+1} = \gamma + \theta y_t + \sigma w_{t+1}
$$

where

- $ w_{t+1} $ is distributed `Normal(0,1)`  
- $ \gamma=1, \sigma=1, y_0 = 0 $  
- $ \theta \in \Theta \equiv \{0.8, 0.9, 0.98\} $  


Given these parameters

- Simulate a single $ y_t $ series for each $ \theta \in \Theta $
  for $ T = 150 $.  Feel free to experiment with different $ T $  
- Overlay plots of the rolling mean of the process for each $ \theta \in \Theta $,
  i.e. for each $ 1 \leq \tau \leq T $ plot  


$$
\frac{1}{\tau}\sum_{t=1}^{\tau}y_T
$$

- Simulate $ N=200 $ paths of the stochastic process above to the $ T $,
  for each $ \theta \in \Theta $, where we refer to an element of a particular
  simulation as $ y^n_t $  
- Overlay plots a histogram of the stationary distribution of the final
  $ y^n_T $ for each $ \theta \in \Theta $.  Hint: pass `alpha`
  to a plot to make it transparent (e.g. `histogram(vals, alpha = 0.5)`) or
  use `stephist(vals)` to show just the step function for the histogram  
- Numerically find the mean and variance of this as an ensemble average, i.e.
  $ \sum_{n=1}^N\frac{y^n_T}{N} $ and
  $ \sum_{n=1}^N\frac{(y_T^n)^2}{N} -\left(\sum_{n=1}^N\frac{y^n_T}{N}\right)^2 $  


Later, we will interpret some of these in [this lecture](https://lectures.quantecon.org/jl/lln_clt.html#)

### Exercise 3

Let the data generating process for a variable be

$$
y = a x_1 + b x_1^2 + c x_2 + d + \sigma w
$$

where $ y, x_1, x_2 $ are scalar observables, $ a,b,c,d $ are parameters to estimate, and $ w $ are iid normal with mean 0 and variance 1

First, let’s simulate data we can use to estimate the parameters

- Draw $ N=50 $ values for $ x_1, x_2 $ from iid normal distributions  


Then, simulate with different $ w $
* Draw a $ w $ vector for the `N` values and then `y` from this simulated data if the parameters were $ a = 0.1, b = 0.2 c = 0.5, d = 1.0, \sigma = 0.1 $
* Repeat that so you have `M = 20` different simulations of the `y` for the `N` values

Finally, calculate order least squares manually (i.e., put the observables
into matrices and vectors, and directly use the equations for
[OLS](https://en.wikipedia.org/wiki/Ordinary_least_squares) rather than a package)

- For each of the `M=20` simulations, calculate the OLS estimates for $ a, b, c, d, \sigma $  
- Plot a histogram of these estimates for each variable  

## Solutions

### Exercise 1

Here’s the iterative approach

In [138]:
function compute_asymptotic_var(A, Σ;
                                S0 = Σ * Σ',
                                tolerance = 1e-6,
                                maxiter = 500)
    V = Σ * Σ'
    S = S0
    err = tolerance + 1
    i = 1
    while err > tolerance && i ≤ maxiter
        next_S = A * S * A' + V
        err = norm(S - next_S)
        S = next_S
        i += 1
    end
    return S
end

compute_asymptotic_var (generic function with 1 method)

In [139]:
A = [0.8  -0.2;
     -0.1  0.7]

Σ = [0.5 0.4;
     0.4 0.6]

2×2 Array{Float64,2}:
 0.5  0.4
 0.4  0.6

Note that all eigenvalues of $ A $ lie inside the unit disc

In [140]:
maximum(abs, eigvals(A))

0.9

Let’s compute the asymptotic variance

In [141]:
our_solution = compute_asymptotic_var(A, Σ)

2×2 Array{Float64,2}:
 0.671228  0.633476
 0.633476  0.858874

Now let’s do the same thing using QuantEcon’s `solve_discrete_lyapunov()` function and check we get the same result

In [142]:
using QuantEcon

In [143]:
norm(our_solution - solve_discrete_lyapunov(A, Σ * Σ'))

3.883245447999784e-6